# DATASCI W261: Machine Learning at Scale 
** W261-4 Spring 2016   
Assignment 13   
April 24, 2016**

Student name **HUSSEIN DANISH**

In [1]:
labVersion = 'MIDS_MLS_HW_13'

In [2]:
%matplotlib inline

In [3]:
import os
import sys
spark_home = os.environ['SPARK_HOME'] = '/home/hdanish/Downloads/spark-1.6.1-bin-hadoop2.6/'

if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.1
      /_/

Using Python version 2.7.11 (default, Dec  6 2015 18:08:32)
SparkContext available as sc, HiveContext available as sqlContext.


In [4]:
import json
import time

### ** HW 13.1: Spark implementation of basic PageRank **

Write a basic Spark implementation of the iterative PageRank algorithm that takes sparse adjacency lists as input.

Make sure that your implementation utilizes teleportation (1-damping/the number of nodes in the network), and further, distributes the mass of dangling nodes with each iteration so that the output of each iteration is correctly normalized (sums to 1). [NOTE: The PageRank algorithm assumes that a random surfer (walker), starting from a random web page, chooses the next page to which it will move by clicking at random, with probability d, one of the hyperlinks in the current page. This probability is represented by a so-called ‘damping factor’ d, where d ∈ (0, 1). Otherwise, with probability (1 − d), the surfer jumps to any web page in the network. If a page is a dangling end, meaning it has no outgoing hyperlinks, the random surfer selects an arbitrary web page from a uniform
distribution and “teleports” to that page]

In your Spark solution, please use broadcast variables and caching to make sure your code is as efficient as possible.

As you build your code, use the test data

s3://ucb-mids-mls-networks/PageRank-test.txt

Or under the Data Subfolder for HW7 on Dropbox with the same file name. 

(On Dropbox https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0)

with teleportation parameter set to 0.15 (1-d, where d, the damping factor is set to 0.85), and crosscheck
your work with the true result, displayed in the first image
in the Wikipedia article:

https://en.wikipedia.org/wiki/PageRank

and here for reference are the corresponding PageRank probabilities:

A,0.033
B,0.384
C,0.343
D,0.039
E,0.081
F,0.039
G,0.016
H,0.016
I,0.016
J,0.016
K,0.016

Run this experiment locally first. Report the local configuration that you used and how long in minutes and seconds it takes to complete your job.

Repeat this experiment on AWS. Report the AWS cluster configuration that you used and how long in minutes and seconds it takes to complete your job. (in your notebook, cat the cluster config file)

In [8]:
# Filip Krunic implementation
# Parse 
def parseRaw(line):    
    """ This function parses the graph elements from the dictionary."""
    
    # Separate
    array = line.split('\t')
    node, neighbors = array 
    neighbors = eval(neighbors)
    
    # Emit
    for k in neighbors.keys(): 
        yield (node, [k])
        yield (k, [])
        

# Calculate PageRank 
def emitPR(line):    
    # Unpack
    node, prTuple = line
    prList, rank = prTuple
    
    # Emit 
    for neighbor in prList: 
        yield (neighbor, rank / len(prList))
        
        # Danglers 
        yield (node, 0)
        
        
# Compute PR with Teleport 
def dampenedPR(line, danglerLoss, totalDanglers, totalNodes, d=0.85):        
    # Unpack
    node, PR = line 
    
    # Update 
    PR *= d
    PR += (1 - d + d * danglerLoss / totalDanglers ) / totalNodes
    
    # Emit 
    return (node, PR)    
        
        
######## INITIALIZE #########
def initPageRank(infile, outfile, n=10):
    # Load 
    graphData = sc.textFile(infile).flatMap(lambda x: parseRaw(x)).reduceByKey(lambda a, b: a + b).cache()
    rank = graphData.map(lambda x: (x[0], 1))

    # Dangling 
    danglers = graphData.filter(lambda x: not bool(x[1]))
    totalDanglers = danglers.count()

    # Diag
    totalNodes = graphData.count()

    # PageRank
    for i in range(n):
        # Neighbor contributions 
        PR = graphData.join(rank).flatMap(lambda x: emitPR(x)).reduceByKey(lambda a, b: a + b)

        # Dangling PR
        danglerPR = PR.join(danglers).map(lambda x: (x[0], x[1][0]))
        danglerLoss = danglerPR.map(lambda x: x[1]).sum()

        # Dampening
        dPR = PR.map(lambda x: dampenedPR(x, danglerLoss, totalDanglers, totalNodes))

        # Normalize 
        totalWeight = dPR.map(lambda x: x[1]).sum()
        nPR = dPR.map(lambda x: (x[0], x[1] / totalWeight))

        # Cycle 
        rank = nPR

    # Results 
    rank.saveAsTextFile(outfile)

In [9]:
startTime = time.time()
initPageRank('PageRank-test.txt', 'pr10-2')
endTime = time.time()
print '\n'
print '10 iterations took ' + str(endTime - startTime) + " seconds"



10 iterations took 47.5687220097 seconds


In [12]:
!cat ./pr10-2/part-* > ./pr10-2.txt
!head -n 11 ./pr10-2.txt

(u'B', 0.3910656743458547)
(u'G', 0.015252405206487117)
(u'J', 0.015252405206487117)
('A', 0.03138768904445686)
(u'I', 0.015252405206487117)
(u'D', 0.03744461211710654)
('C', 0.3491026787812407)
(u'K', 0.015252405206487117)
('F', 0.03744461211710654)
('E', 0.07729270756179903)
(u'H', 0.015252405206487117)


###HW 13.3: Spark GraphX versus your implementation of PageRank

Run the Spark  GraphX PageRank implementation on the Wikipedia dataset for 10 iterations,
and display the top 100 ranked nodes (with alpha = 0.85).

Run your PageRank implementation on the Wikipedia dataset for 50 iterations,
and display the top 100 ranked nodes (with teleportation factor of 0.15). 
Have the top 100 ranked pages changed? Comment on your findings. Plot both 100 curves.

Report the AWS cluster configuration that you used and how long in minutes and seconds it takes to complete this job.

Put the runtime results of HW13.2 and HW13.3 in a tabular format (with rows corresponding to implemention and columns corresponding to experiment setup (10 iterations, 50 iterations)). Discuss the run times and explaing the differences. 

Plot the pagerank values for the top 100 pages resulting from the 50 iterations run (using GraphX). Then plot the pagerank values for the same 100 pages that resulted from the 50 iterations run of your homegrown pagerank implemnentation.  Comment on your findings.  Have the top 100 ranked pages changed? Have the pagerank values changed? Explain.

In [25]:
# First trying this out on the toy set
# We want to parse the input to have a file with format "NODE NEIGHBOUR" on each line
def graphxParse(line):
    node, links = line.split('\t')
    for link in eval(links).keys():
        yield node + ' ' + link

In [26]:
# Try out this function
for e in graphxParse("5	{'4': 1, '2': 1, '6': 1}"):
    print e

5 2
5 4
5 6


In [27]:
# Now we want to create our input file
def graphxInit(infile, outfile):
    sc.textFile(infile).flatMap(lambda x: graphxParse(x)).saveAsTextFile(outfile)
    
graphxInit('PageRank-test_indexed.txt', 'toygraph')

In [30]:
!cat ./toygraph/part-* > ./toygraph.txt
!cat ./toygraph.txt

2 3
3 2
4 1
4 2
5 2
5 4
5 6
6 2
6 5
7 2
7 5
8 2
8 5
9 2
9 5
10 5
11 5


In [34]:
%%writefile pagerank.scala
import org.apache.spark._
import org.apache.spark.graphx._

object HWPageRank {
    def main(args: Array[String]) {
        val conf = new SparkConf().setAppName("pagerank")
        val sc = new SparkContext(conf)
        
        val graph = GraphLoader.edgeListFile(sc, "/home/hdanish/Documents/UCB/W261/Week13/toygraph.txt")

        val ranks = graph.staticPageRank(10).vertices
        
        val top100 = ranks.sortBy(_._2, false).take(100)
        sc.parallelize(top100).saveAsTextFile("pr10g-1")        
    }
}

Overwriting pagerank.scala


In [33]:
%%writefile build.sbt
name := "pagerank"
version := "1.0"
scalaVersion := "2.10.5"
libraryDependencies ++= Seq("org.apache.spark" %% "spark-core" % "1.6.1", "org.apache.spark" %% "spark-graphx" % "1.6.1")

Writing build.sbt


Directory layout should look like this

$ find .

.

./build.sbt

./src

./src/main

./src/main/scala

./src/main/scala/pagerank.scala

**So need to create subdirectories src, src/main, src/main/scala and then put the scala file in the last subdirectory**

Then we can run the next steps which is "sbt package" to compile

In [45]:
!sbt package

[info] Set current project to pagerank (in build file:/home/hdanish/Documents/UCB/W261/Week13/)
[success] Total time: 3 s, completed 28-Apr-2016 8:28:04 PM


In [42]:
!/home/hdanish/Downloads/spark-1.6.1-bin-hadoop2.6//bin/spark-submit --class "HWPageRank" --master local[4] \
 target/scala-2.10/pagerank_2.10-1.0.jar

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
16/04/28 19:56:13 INFO SparkContext: Running Spark version 1.6.1
16/04/28 19:56:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/04/28 19:56:14 WARN Utils: Your hostname, hdanish-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface eth0)
16/04/28 19:56:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
16/04/28 19:56:14 INFO SecurityManager: Changing view acls to: hdanish
16/04/28 19:56:14 INFO SecurityManager: Changing modify acls to: hdanish
16/04/28 19:56:14 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users with view permissions: Set(hdanish); users with modify permissions: Set(hdanish)
16/04/28 19:56:15 INFO Utils: Successfully started service 'sparkDriver' on port 56027.
16/04/28 19:56:16 INFO Slf4jLogger: Slf4jLogger starte

In [44]:
!cat ./pr10g-1/part-* > ./pr10g-1.txt
!cat ./pr10g-1.txt

(2,2.837824310201201)
(3,2.4868526601543555)
(5,0.7503400819355781)
(4,0.3625952805244703)
(6,0.3625952805244703)
(1,0.3040900819355781)
(8,0.15)
(10,0.15)
(11,0.15)
(7,0.15)
(9,0.15)
